In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'sota_data.csv')
df.head()

,serial_number,model,capacity_bytes,hard_live_cost,smart_1_normalized,smart_2_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_6_normalized,...,smart_246_raw,smart_247_raw,smart_248_raw,smart_249_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_253_raw,smart_254_raw,smart_255_raw
0,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,2088,100.0,134.0,139.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,2088,100.0,133.0,127.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,2088,100.0,134.0,124.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,2088,100.0,135.0,121.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,2088,100.0,135.0,123.0,100.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['serial_number', 'model', 'capacity_bytes', 'hard_live_cost',
       'smart_1_normalized', 'smart_2_normalized', 'smart_3_normalized',
       'smart_4_normalized', 'smart_5_normalized', 'smart_6_normalized',
       ...
       'smart_246_raw', 'smart_247_raw', 'smart_248_raw', 'smart_249_raw',
       'smart_250_raw', 'smart_251_raw', 'smart_252_raw', 'smart_253_raw',
       'smart_254_raw', 'smart_255_raw'],
      dtype='object', length=514)

In [4]:
data = df

In [5]:
from sklearn.model_selection import train_test_split


data = df
data = data.drop(columns=['serial_number'])
threshold = 0.5  # Порог: 50%
data = data.loc[:, data.isnull().mean() < threshold]

X = data.drop(columns=['hard_live_cost'])
y = data['hard_live_cost']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
data.isna().sum()

model                        0
capacity_bytes               0
hard_live_cost               0
smart_1_normalized         554
smart_2_normalized      116781
smart_3_normalized        3075
smart_4_normalized        3075
smart_5_normalized        2318
smart_7_normalized        3075
smart_8_normalized      116781
smart_9_normalized         441
smart_10_normalized       3075
smart_12_normalized        441
smart_192_normalized      1198
smart_193_normalized      3470
smart_194_normalized       441
smart_196_normalized    115974
smart_197_normalized      8397
smart_198_normalized      2431
smart_199_normalized      2318
smart_240_normalized     79115
smart_241_normalized    116036
smart_242_normalized    116038
smart_1_raw                554
smart_2_raw             116781
smart_3_raw               3075
smart_4_raw               3075
smart_5_raw               2318
smart_7_raw               3075
smart_8_raw             116781
smart_9_raw                441
smart_10_raw              3075
smart_12

In [7]:
from autogluon.tabular import TabularPredictor
# Разделение данных на обучающие и тестовые
data = data.sparse.to_dense()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Повторное обучение на обучающих данных
predictor = TabularPredictor(
    label = 'hard_live_cost',
    problem_type = 'regression',
    eval_metric = 'mean_absolute_error',
    path='weights_model'
    ).fit(train_data,  time_limit = 600, presets = 'best_quality')

# Оценка модели на тестовой выборке
performance = predictor.evaluate(test_data)
print(performance)

c:\Users\Goldian\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20240907_164728"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       20.37 GB / 31.95 GB (63.7%)
Disk Space Avail:   14.94 GB / 232.24 GB (6.4%)
Presets specified: ['interpretable']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels\ag-20240907_164728"
Train Data Rows:    194135
Train Data Columns: 42
Label Column:       hard_live_cost
Problem Type:       regression
Preprocessing data ...
Usin

{'symmetric_mean_absolute_percentage_error': -0.006090034499772724, 'root_mean_squared_error': -122.04190074696947, 'mean_squared_error': -14894.225537933147, 'mean_absolute_error': -15.089733025531231, 'r2': 0.20348221063613892, 'pearsonr': 0.4825216288844454, 'median_absolute_error': -2.6044921875}


c:\Users\Goldian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [25]:
from autogluon.tabular import TabularPredictor
import pandas as pd

INPUT_DATA = r'sota_data.csv'
THRESH = 0.5

data = pd.read_csv(INPUT_DATA)
data = data.drop(columns=['serial_number'])
data = data.loc[:, data.isnull().mean() < THRESH]
for column in data.columns:
    if hasattr(data[column], "sparse") and data[column].sparse is not None:
        data[column] = data[column].sparse.to_dense()

predictor = TabularPredictor(
    label = 'hard_live_cost',
    problem_type = 'regression',
    eval_metric = 'mean_absolute_error',
    path='weights_model',
    ).fit(data, time_limit = 600, presets = 'best_quality', keep_only_best=True)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       21.34 GB / 31.95 GB (66.8%)
Disk Space Avail:   14.95 GB / 232.24 GB (6.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up t

In [26]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-10.494457,mean_absolute_error,126.673415,261.555526,0.002998,0.792331,3,True,11
1,LightGBM_BAG_L2,-10.510597,mean_absolute_error,124.526661,162.514602,0.960175,7.923994,2,True,8
2,RandomForestMSE_BAG_L1,-10.599180,mean_absolute_error,2.838665,71.654275,2.838665,71.654275,1,True,5
3,CatBoost_BAG_L2,-10.621635,mean_absolute_error,123.710519,199.148334,0.144033,44.557726,2,True,10
4,RandomForestMSE_BAG_L2,-10.638367,mean_absolute_error,125.566209,208.281475,1.999723,53.690868,2,True,9
5,LightGBM_BAG_L1,-11.005176,mean_absolute_error,1.880066,9.276416,1.880066,9.276416,1,True,4
6,CatBoost_BAG_L1,-11.588299,mean_absolute_error,0.134040,56.953635,0.134040,56.953635,1,True,6
7,LightGBMXT_BAG_L1,-11.651024,mean_absolute_error,4.242972,12.040605,4.242972,12.040605,1,True,3
8,KNeighborsDist_BAG_L1,-13.409407,mean_absolute_error,57.017465,0.336635,57.017465,0.336635,1,True,2
9,KNeighborsUnif_BAG_L1,-13.453863,mean_absolute_error,56.598976,0.353124,56.598976,0.353124,1,True,1


2024-09-08 02:00:21,008	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-09-08 02:00:21,023	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-09-08 02:00:21,025	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-09-08 02:00:21,028	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.


In [28]:
INPUT_DATA_SUB = r'data_example\data_Q1_2023\2023-01-01.csv'
predict_data = pd.read_csv(INPUT_DATA_SUB)
predict_data


,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-01-01,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,0,100.0,0.0,134.0,101.0,139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,133.0,104.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-01,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,134.0,102.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235592,2023-01-01,3WJRXE8K,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235593,2023-01-01,3WJXK5YK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235594,2023-01-01,3WJZY4EJ,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235595,2023-01-01,3WK6NENK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
loaded_predictor = TabularPredictor.load('weights_model')

predict_data = pd.read_csv(INPUT_DATA_SUB)

s_number = predict_data['serial_number']
predict_data = predict_data.drop(columns=['date', 'serial_number'])

predictions = loaded_predictor.predict(predict_data)

print(predictions)

KeyboardInterrupt: 

In [57]:
predict_data['predicted_days_to_failure'] = predictions
predict_data['serial_number'] = s_number
predict_data = predict_data[['model', 'capacity_bytes', 'predicted_days_to_failure', 'serial_number']]

C:\Users\Goldian\AppData\Local\Temp\ipykernel_19396\2112857546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['predicted_days_to_failure'] = predictions


In [58]:
predict_data

,model,capacity_bytes,predicted_days_to_failure,serial_number
0,HGST HDS5C4040ALE630,4000787030016,2086.920410,PL1331LAGDJ5GH
1,HGST HMS5C4040BLE640,4000787030016,2086.929688,PL2331LAGLW5UJ
2,HGST HMS5C4040BLE640,4000787030016,2086.919189,PL2331LAGM410J
3,HGST HMS5C4040BLE640,4000787030016,2086.919678,PL2331LAGM5NTJ
4,HGST HMS5C4040BLE640,4000787030016,2086.915039,PL2331LAGM5R4J
...,...,...,...,...
235592,WDC WUH721816ALE6L4,16000900661248,2049.384521,3WJRXE8K
235593,WDC WUH721816ALE6L4,16000900661248,2049.337402,3WJXK5YK
235594,WDC WUH721816ALE6L4,16000900661248,2049.395752,3WJZY4EJ
235595,WDC WUH721816ALE6L4,16000900661248,2049.334473,3WK6NENK


In [80]:
predict_data['predicted_months_to_failure'] = predict_data['predicted_days_to_failure'] / 30

# 3. Определение временных интервалов
def assign_time_interval(months):
    if months <= 3:
        return '0-3 месяца'
    elif months <= 6:
        return '4-6 месяцев'
    elif months <= 9:
        return '7-9 месяцев'
    elif months <= 12:
        return '10-12 месяцев'
    else:
        return 'Более 12 месяцев'

predict_data['time_interval'] = predict_data['predicted_months_to_failure'].apply(assign_time_interval)

# 4. Группировка данных и подсчет количества дисков
# Фильтруем только те диски, которые выйдут из строя в течение следующих 12 месяцев
data_filtered = predict_data[predict_data['time_interval'] != 'Более 12 месяцев']

# Группируем данные
result = data_filtered.groupby(
    ['capacity_bytes', 'model', 'time_interval']
).size().reset_index(name='disk_count')

# Упорядочиваем интервалы времени
interval_order = ['0-3 месяца', '4-6 месяцев', '7-9 месяцев', '10-12 месяцев']
result['time_interval'] = pd.Categorical(
    result['time_interval'],
    categories=interval_order,
    ordered=True
)
# result['capacity_tbytes'] = result['capacity_bytes'] // 10**12

# 5. Вывод результата
pd.DataFrame(result, index=[i for i in range(len(result))]).sort_values(by=['time_interval', 'capacity_bytes', 'model'])

,capacity_bytes,model,time_interval,disk_count
3,4000787030016,ST4000DM000,0-3 месяца,4
6,8001563222016,ST8000DM002,0-3 месяца,3
8,8001563222016,ST8000NM0055,0-3 месяца,3
18,12000138625024,ST12000NM0008,0-3 месяца,3
25,14000519643136,ST14000NM0138,0-3 месяца,1
28,14000519643136,TOSHIBA MG07ACA14TA,0-3 месяца,1
33,16000900661248,ST16000NM001G,0-3 месяца,2
36,16000900661248,TOSHIBA MG08ACA16TEY,0-3 месяца,1
0,500107862016,TOSHIBA MQ01ABF050,4-6 месяцев,2
7,8001563222016,ST8000DM002,4-6 месяцев,1


In [46]:
(predictions < 0).sum()

9

In [23]:
# Удаление всех моделей, кроме лучшей
predictor.delete_models(models_to_keep='best')

Dry run enabled, AutoGluon would have deleted the following models: ['RuleFit']
	Directory AutogluonModels\ag-20240907_164728\models\RuleFit would have been deleted.
To perform the deletion, set dry_run=False


In [22]:
predictor.save(r'C:\fast_dev\demand-hardware-failure\weights_model/')

In [18]:
print(predictor.path)

AutogluonModels\ag-20240907_164728


In [14]:
loaded_predictor = TabularPredictor.load('weights_model')

This means that the predictor was fit in an AutoGluon version `<=0.3.1`.


FileNotFoundError: [Errno 2] No such file or directory: 'weights_model\\predictor.pkl'

In [12]:
loaded_predictor = TabularPredictor.load('weights_model')

# Использование загруженной модели для предсказаний
predictions = loaded_predictor.predict(test_data)
print(predictions)

This means that the predictor was fit in an AutoGluon version `<=0.3.1`.


FileNotFoundError: [Errno 2] No such file or directory: 'weights_model\\predictor.pkl'

In [9]:
leaderboard = predictor.leaderboard(test_data, silent=True)
print(leaderboard)

       model  score_test  score_val                               eval_metric  \
0  RuleFit_2   -0.006090  -0.005058  symmetric_mean_absolute_percentage_error   
1    RuleFit   -0.006301  -0.005342  symmetric_mean_absolute_percentage_error   

   pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  \
0        0.080019       0.017005  1020.833917                 0.080019   
1        0.081017       0.011007   355.715345                 0.081017   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.017005        1020.833917            1       True   
1                0.011007         355.715345            1       True   

   fit_order  
0          2  
1          1  


c:\Users\Goldian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Goldian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [8]:
print(performance)

{'symmetric_mean_absolute_percentage_error': -0.006090034499772724, 'root_mean_squared_error': -122.04190074696947, 'mean_squared_error': -14894.225537933147, 'mean_absolute_error': -15.089733025531231, 'r2': 0.20348221063613892, 'pearsonr': 0.4825216288844454, 'median_absolute_error': -2.6044921875}


In [7]:
from catboost import CatBoostRegressor

categorical_features = ['model']
# Инициализация модели
model = CatBoostRegressor(iterations=100, 
                          learning_rate=0.1, 
                          depth=12, 
                          loss_function='MAE', 
                          random_seed=42)

# Обучение модели
model.fit(X_train, y_train, 
          cat_features=categorical_features, 
          eval_set=(X_test, y_test),
          verbose=100,
          early_stopping_rounds=50)


0:	learn: 9.6917450	test: 10.6045881	best: 10.6045881 (0)	total: 521ms	remaining: 51.6s


KeyboardInterrupt: 